In [15]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## Cohort

In [16]:
t = pd.read_sas("../Data Validation/BEST/t.sas7bdat", index='BEST_ID').sort_index()
pcsf = pd.read_sas("../Data Validation/BEST/pcsf.sas7bdat", index='BEST_ID').sort_index()
pe2 = pd.read_sas("../Data Validation/BEST/pe2.sas7bdat", index='BEST_ID').sort_index()
pe = pd.read_sas("../Data Validation/BEST/pe.sas7bdat", index='BEST_ID').sort_index()
cvs = pd.read_sas("../Data Validation/BEST/cvs.sas7bdat", index='BEST_ID').sort_index()
cvh1 = pd.read_sas("../Data Validation/BEST/cvh_1.sas7bdat", index='BEST_ID').sort_index()
cvh2 = pd.read_sas("../Data Validation/BEST/cvh_2.sas7bdat", index='BEST_ID').sort_index()
ecg = pd.read_sas("../Data Validation/BEST/ecg.sas7bdat", index='BEST_ID').sort_index()
ecg2 = pd.read_sas("../Data Validation/BEST/ecg2.sas7bdat", index='BEST_ID').sort_index()
xp = pd.read_sas("../Data Validation/BEST/xp.sas7bdat", index='BEST_ID').sort_index()
xp2 = pd.read_sas("../Data Validation/BEST/xp2.sas7bdat", index='BEST_ID').sort_index()
lab12 = pd.read_sas("../Data Validation/BEST/lab_12.sas7bdat", index='BEST_ID').sort_index()
clab12 = pd.read_sas("../Data Validation/BEST/clab_12.sas7bdat", index='BEST_ID').sort_index()

In [17]:
# Started to split list into tx and placebo at beginning then realized things could misalign based
# on how we're relying on sorting.  Will build one dataframe then split into 2 at the end.
cohort = pd.DataFrame()
cohort['ID'] = t.index
cohort = cohort.set_index('ID')

#drop_lst = []
#for idx in cohort_pl.index:
#    pt = t.loc[idx]
#    if pt['GROUP'] == 2:
#        drop_lst.append(idx)
#cohort_pl.drop(drop_lst, inplace=True)

#Tx dataframe
#cohort_tx = pd.DataFrame()
#cohort_tx['ID'] = t.index
#cohort_tx = cohort_tx.set_index('ID')

#drop_lst = []
#for idx in cohort_tx.index:
#    pt = t.loc[idx]
#    if pt['GROUP'] == 1:
#        drop_lst.append(idx)
#cohort_tx.drop(drop_lst, inplace=True)

#lst_pl = []
#lst_tx = []
#for idx in t.index:
#    pt = t.loc[idx]
#    if pt['GROUP'] == 1:
#        lst_pl.appendf

cohort['Age'] = pcsf['PCAGE'].tolist()

lst = []
for idx in pcsf.index:
    pt = pcsf.loc[idx]
    if pt['PCSEX'] == 1:
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#print(pcsf['RACE4'].value_counts())
#print('b\'White\' - test') --- My string appears to perfectly match the elements in the field but
# when I check the values afterwards everyone is categorized as 2 and there are no ones.  Tried using
# is instead of ==, no difference. -- solution - use .decode on byte object
lst = []
for idx in pcsf.index:
    pt = pcsf.loc[idx]
    if pt['RACE4'].decode("utf-8") == 'White':
        lst.append(1)
    else:
        lst.append(2)

#weight, convert to all be Kg -- i used 2nd visit because it had more indices - more complete
lst = []
for idx in t.index:
    if idx not in pe.index:
        lst.append(np.nan)
    else:
    #    print(pe.index)
    #    pt = pe.loc[idx][2.0]
    #    lst.append(pt['PEWT'] * 0.45359237)
        pt = pe.loc[idx]
        try:
            #print(pt[pt['VISIT']])
            pt = pt[pt['VISIT'] == 2.0]
            wt = pt['PEWT'] * 0.45359237
            lst.append(wt[idx])
        except:
            lst.append(None)
        
cohort['Wt_B'] = lst
#

#lst = [] -- pe2 has weight but which visit do we use as a discharge?  Find the latest visit for
#each recorded pt?
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA20B1'] == 'L': #convert lbs to Kg
#        lst.append(pt['HFAA20B'] * 0.45359237)
#    else:
#        lst.append(pt['HFAA20B'])

#cohort['Wt_D'] = lst

#cohort['BMI_D'] = hfcoccps16['BMI'].tolist() --- no bmi found
#cohort['EjF'] = hfcoccps16['LVEF_CUR'].tolist() -- no feature found



#NYHA
lst = []
for idx in t.index:
    if idx not in cvs.index:
        lst.append(np.nan)
    else:
        pt = cvs.loc[idx]
        try:

            pt = pt[pt['VISIT'] == 1.0]
            wt = pt['CVNYH']
            lst.append(wt[idx])
        except:
            lst.append(None)
cohort['NYHA'] = lst

# days = hfcchips16['CHIA0C_DAYS'] - hfcchips16['CHIA6A_DAYS']  There are many Days to visit fields in
# the forms.  I'm skimmed the protocols.  Don't know if there's a way to use these to get hosp days.
# cohort['TotalHospDays'] = days.tolist()

#cohort['TotalHospDays'] = hfcoccps16['LOS'].tolist()

cohort.head(5)

,Age,Gender,Wt_B,NYHA
ID,,,,
1.0,73.0,2,49.895161,3.0
2.0,72.0,1,74.842741,3.0
3.0,58.0,1,97.522360,3.0
4.0,42.0,1,135.624119,3.0
5.0,58.0,1,69.853225,4.0


In [18]:
#conditions, Ischemic -- couldn't find
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA6A'] == 'Y' or pt['HFAA6A'] == 'Y' or pt['HFAA11K'] == 'Y' or  pt['HFAA16J']== 'Y' or  pt['HFAA76']== 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['ISCH'] = lst

#Nonischemic -- couldnt find
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA6B'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['NonISCH'] = lst

#AF -- fib and flutter
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVAFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#AlchE -- cvh_2.sas has CVETOH - alcoholic cardiomyopathy - not sure if this is equivalent here.
# also found br.sas BRABUSE but combines alcohol and drug abuse
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA9B'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['AlchE'] = lst

#Angina
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVANG'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#ARRH - cvh1 CVAFIB and cvh1 CVVFIB, found no  vent tachy. pcavblk on pcsf has all no values
# so was excluded, ecg and ecg2 are the results of ecg tests.  There is a ECRH feature where number 3
# correspond to sinus bradycardia per entire forms and coding manual.pdf. ecg has many observations, ecg2
# not many.  I don't know if we want to categorize some patients as having bradycardia if we have one
# ecg of sinus bradycardia, they may not be symptomatic.

lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVAFIB'] == 1.0 or pt['CVVFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['ARRH'] = lst

#CARREST -- couldn't find field
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA11D'] == 'Y' or pt['HFAA74'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['CARREST'] = lst

#COPD -- didn't find - found smoking hx - surprised.
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA10B'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['COPD'] = lst

#Depr -- also didn't find
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA14B'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['DEPR'] = lst

#Diab
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVHXDIAB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#HEPT -- pe PEHEP, didn't find hepatojugular reflux or liver disease
lst = []
for idx in t.index:
    #if idx not in pe.index:
    #    lst.append(np.nan)
    #else:
    pt = pe.loc[idx]
    try:
        pt = pt[pt['VISIT'] == 2.0]
        wt = pt['PEHEP']
        if wt[idx] == 2.0:
            wt[idx] = 0.0
        lst.append(wt[idx])
    except:
        lst.append(None)
cohort['HEPT'] = lst

#HTN -- cvh2 CVHHXHTXN , didn't find pulm htn
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVHXHTXN'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig -- didn't find
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA9E'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['MALIG'] = lst

#Renal - didn't find
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA13A'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['RENAL'] = lst

#Smok -- cvh_1.sas - CVYRSMOK - years smoked, if field is more than 0 I flagged as history of smoking.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVYRSMOK'] != 0.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#Stroke TIA - found neither stroke nor transient ischemic attack
#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA14A'] == 'Y' or pt['HFAA75'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['StrokeTIA'] = lst

#VAHD -- because of cvduratn every pt falls into this category as the trial was testing folks that have
#chf
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVPVD'] == 1.0 or pt['CVDURATN'] != 0.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

#VF -- doesnt include vtach.  Couldnt find it.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVVFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD -- cvvalrep has 0 and 2, cvaortic has 0 and 2
lst = []
for idx in t.index:
    pt_cvh1 = cvh1.loc[idx]
    pt_cvh2 = cvh2.loc[idx]
    if pt_cvh1['CVVALVUL'] == 1.0 or pt_cvh1['CVVALREP'] == 1.0 or pt_cvh2['CVAORTIC'] == 1.0 or pt_cvh2['CVMITRAL'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT -- found no v tach feature, this feature is essentially the same as VF.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVVFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVBYPAS'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#HTRANS -- the xp dataset has some features about transplantation.  I assume they are referring to heart
# transplants but the form doesn't make it clear who has a transplant and who doesn't.  It has new transplant
# status and has either listed or transplanted.  Listed may mean they are waiting for heart and trans
# may mean they have gotten it. Only 5 in xp2 so added manually.
lst = []
for idx in t.index:
    if idx not in xp.index:
        lst.append(np.nan)
    else:
        pt= xp.loc[idx]
        #pt_xp2 = xp2.loc[idx]
        try:
            pt= pt[pt['XPNUM'] == 552.0]
            wt= pt['XPSTA']
            #pt_xp2 = pt_xp2[pt_xp2['XPNUM'] == 552.0]
            #wt_xp2 = pt_xp2['XPSTA']
            if wt[idx] == 2.0:
                wt[idx] = 1.0
            else:
                wt[idx] = 0.0
            lst.append(wt[idx])
        except:
            lst.append(None)
cohort['HTRANS'] = lst
cohort['HTRANS'].loc[213.0] = 1.0
cohort['HTRANS'].loc[295.0] = 1.0
cohort['HTRANS'].loc[778.0] = 1.0
cohort['HTRANS'].loc[1438.0] = 1.0
cohort['HTRANS'].loc[2649.0] = 1.0


#ICD
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVDEFIBR'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#Pace
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVPACEMK'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI -- apparently PTCA is the same procedure.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVANGIOP'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst


cohort.head(5)

,Age,Gender,Wt_B,NYHA,AF,ANGP,ARRH,DIAB,HEPT,HTN,SMOKING,VAHD,VF,VHD,VT,CABG,HTRANS,ICD,PACE,PTCI
ID,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2,49.895161,3.0,0,0,1,0,0.0,0,0,1,1,0,1,0,NaN,1,0,0
2.0,72.0,1,74.842741,3.0,1,1,1,0,0.0,1,0,1,0,0,0,1,NaN,0,0,0
3.0,58.0,1,97.522360,3.0,0,0,0,1,0.0,1,1,1,0,0,0,0,NaN,0,0,0
4.0,42.0,1,135.624119,3.0,0,1,0,0,1.0,1,1,1,0,0,0,0,NaN,0,0,0
5.0,58.0,1,69.853225,4.0,0,1,0,0,0.0,1,1,1,0,0,0,0,NaN,0,0,1


In [19]:
# Labs

#cohort['BUN_D'] = hfchfaps16['HFAA45B'].tolist()
lst = []
for idx in t.index:
    if idx not in lab12.index:
        lst.append(np.nan)
    else:
        pt = lab12.loc[idx]
        lst.append(pt['LABUN'])
cohort['BUN_D'] = lst


#CRT_D
lst = []
for idx in t.index:
    if idx not in lab12.index:
        lst.append(np.nan)
    else:
        pt = lab12.loc[idx]
        lst.append(pt['LACRE'])
cohort['CRT_D'] = lst

#HEC_D -- only 9 observations
#cohort['HEC_D'] = hfchfaps16['HFAA38B'].tolist()
lst = []
for idx in t.index:
    if idx not in clab12.index:
        lst.append(np.nan)
    else:
        pt = clab12.loc[idx]
        lst.append(pt['CLAHCT'])
cohort['HEC_D'] = lst

#cohort['HEM_D'] = hfchfaps16['HFAA37B'].tolist() -- didn't find hemoglobin
#cohort['SOD_D'] = hfchfaps16['HFAA43B'].tolist() -- discharge has 9 observations. Useful?

#Meds
#lst = []
#for idx in hfchfaps16.index: -- mort has 0/1 feature if pt was on ace inhib 1 week before death?
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA59'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['ACE_B'] = lst

#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA59A'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['ACE_D'] = lst

#lst = [] -- all people in bucindolol group on t could be included. Dont know if non bucindolol group
# was on beta blockers though.
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA65'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['BET_B'] = lst

#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA65A'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['BET_D'] = lst

lst = []
#for idx in hfchfaps16.index: ---- nitrates not found
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA71'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['NIT_B'] = lst

#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA71A'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['NIT_D'] = lst

#lst = [] -- only found iv diuretic, must be inpatient, dont know long term, or reference to death
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA68'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['DIUR_B'] = lst

#lst = []
#for idx in hfchfaps16.index:
#    pt = hfchfaps16.loc[idx]
#    if pt['HFAA68A'] == 'Y':
#        lst.append(1)
#    else:
#        lst.append(0)
#cohort['DIUR_D'] = lst

#cohort

count    9.000000
mean     0.378889
std      0.046488
min      0.320000
25%      0.340000
50%      0.380000
75%      0.410000
max      0.450000
Name: HEC_D, dtype: float64


In [20]:
lst = []
for idx in t.index:
    #if idx not in pe.index:
    #    lst.append(np.nan)
    #else:
    pt = pe.loc[idx]
    try:
        pt = pt[pt['VISIT'] == 2.0]
        wt = pt['PEHR']
        lst.append(wt[idx])
    except:
        lst.append(None)
cohort['HRTRT'] = lst

lst = []
for idx in t.index: # 1 systolic missing
    #if idx not in pe.index:
    #    lst.append(np.nan)
    #else:
    pt = pe.loc[idx]
    try:
        pt = pt[pt['VISIT'] == 1.0]
        wt = pt['PEBPS']
        lst.append(wt[idx])
    except:
        lst.append(None)
cohort['BPSYS_B'] = lst


lst = []
for idx in t.index:# 2 diastolic missing
    #if idx not in pe.index:
    #    lst.append(np.nan)
    #else:
    pt = pe.loc[idx]
    try:
        pt = pt[pt['VISIT'] == 1.0]
        wt = pt['PEBPD']
        lst.append(wt[idx])
    except:
        lst.append(None)
cohort['BPDIAS_B'] = lst

#print(pe2['VISIT'].value_counts()) # discharge values for sys and dia are difficult because it seems
# many patients last visit are recorded under different days.  Would have to find each patients highest
# visit number and get sys and dia from that.
#lst = []
#for idx in t.index: # 1 systolic missing
    #if idx not in pe.index:
    #    lst.append(np.nan)
    #else:
#    pt = pe2.loc[idx]
#    try:
#        pt = pt[pt['VISIT'] == 1.0]
#        wt = pt['PEBPS']
#        lst.append(wt[idx])
#    except:
#        lst.append(None)
#cohort['BPSYS_D'] = lst
#cohort['BPDIAS_D'] = hfchfaps16['HFAA17D'].tolist()

#cohort['RAP'] = hfchfaps16['HFAA31B1'].tolist() ----- didn't find any of these
#cohort['PAS'] = hfchfaps16['HFAA31B2'].tolist()
#cohort['PCWP'] = hfchfaps16['HFAA31B3'].tolist()
#cohort['CO'] = hfchfaps16['HFAA31B4'].tolist()
#cohort['CI'] = hfchfaps16['HFAA31B5'].tolist()


idx = sorted(set(cohort.index))

#MAP
#chg = []
base = []
#final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    #f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    #c = f - b

    base.append(b)
    #final.append(f)
    #chg.append(c)
    
cohort['MAP_B'] = base
#cohort['MAP_D'] = final

#CPI
#chg = []
#base = []
#final = []
#for i in idx:
#    t = cohort.loc[i]

#    b = (t['CI'] * t['MAP_B']) / 451
#    f = (t['CI'] * t['MAP_D']) / 451
#    c = f - b

#    base.append(b)
#    final.append(f)
#    chg.append(c)
    
#cohort['CPI_B'] = base
#cohort['CPI_D'] = final
    
#PP
#chg = []
base = []
#final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
#    f = t['BPSYS_D'] - t['BPDIAS_D']
#    c = f - b

    base.append(b)
#    final.append(f)
#    chg.append(c)
    
cohort['PP_B'] = base
#cohort['PP_D'] = final

#PPP
#chg = []
base = []
#final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
#    f = t['PP_D'] / t['BPSYS_D']
#    c = f - b

    base.append(b)
#    final.append(f)
#    chg.append(c)
    
cohort['PPP_B'] = base
#cohort['PPP_D'] = final

#SVR
#chg = []
#base = []
#final = []
#for i in idx:
#    t = cohort.loc[i]

#    b = 80 * (t['MAP_B']  - t['RAP']) / t['CO']
#    f = 80 * (t['MAP_D']  - t['RAP']) / t['CO']
#    c = f - b

#    base.append(b)
#    final.append(f)
#    chg.append(c)
    
#cohort['SVR_B'] = base
#cohort['SVR_D'] = final

#RAT
#chg = []
#base = []
#final = []
#for i in idx:
#    t = cohort.loc[i]
#    b = t['RAP'] / t['PCWP']

#    base.append(b)

#cohort['RAT'] = base


#PPRatio
'''chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['HRTRT']
    f = t['PP_D'] / t['HRTRT']
    c = f - b

    base.append(b)
    final.append(f)
    
cohort['PPRatio_B'] = base
cohort['PPRatio_D'] = final


#SAPi
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['SAPi_B'] = base
cohort['SAPi_D'] = final

#CPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPDIAS_B'] - t['PCWP']
    f = t['BPDIAS_D'] - t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['CPP_B'] = base
cohort['CPP_D'] = final

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['RAP']
    f = t['PP_D'] / t['RAP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PRAPRat_B'] = base
cohort['PRAPRat_D'] = final'''


cohort.head(5)

,Age,Gender,Wt_B,NYHA,AF,ANGP,ARRH,DIAB,HEPT,HTN,...,PTCI,BUN_D,CRT_D,HEC_D,HRTRT,BPSYS_B,BPDIAS_B,MAP_B,PP_B,PPP_B
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2,49.895161,3.0,0,0,1,0,0.0,0,...,0,NaN,NaN,NaN,68.0,130.0,70.0,176.666667,60.0,0.461538
2.0,72.0,1,74.842741,3.0,1,1,1,0,0.0,1,...,0,NaN,NaN,NaN,60.0,125.0,85.0,181.666667,40.0,0.320000
3.0,58.0,1,97.522360,3.0,0,0,0,1,0.0,1,...,0,NaN,NaN,NaN,104.0,118.0,82.0,172.666667,36.0,0.305085
4.0,42.0,1,135.624119,3.0,0,1,0,0,1.0,1,...,0,NaN,NaN,NaN,79.0,135.0,100.0,201.666667,35.0,0.259259
5.0,58.0,1,69.853225,4.0,0,1,0,0,0.0,1,...,1,NaN,NaN,NaN,72.0,133.0,71.0,180.333333,62.0,0.466165


In [21]:
#Make separate datasets

#HEMO
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
hemoDF

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,0,0,0,0,0,0,0,0,...,60.0,0.461538,0,0,0,0,0,0,0,0
1.0,73.0,2.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0
2.0,72.0,1.0,0,0,0,0,0,0,0,0,...,40.0,0.320000,0,0,0,0,0,0,0,0
2.0,72.0,1.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0
3.0,58.0,1.0,0,0,0,0,0,0,0,0,...,36.0,0.305085,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706.0,78.0,2.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0
2707.0,80.0,1.0,0,0,0,0,0,0,0,0,...,45.0,0.409091,0,0,0,0,0,0,0,0
2707.0,80.0,1.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0


In [22]:
#Get only patients with hemo
hemoTest = hemoDF[['RAP', 'PAS', 'PCWP', 'CO', 'CI']]
hemoComposite = hemoDF.loc[~(np.isnan(hemoTest)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)


(5414, 28)


,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,0,0,0,0,0,0,0,0,...,60.0,0.461538,0,0,0,0,0,0,0,0
1.0,73.0,2.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0
2.0,72.0,1.0,0,0,0,0,0,0,0,0,...,40.0,0.320000,0,0,0,0,0,0,0,0
2.0,72.0,1.0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0,0,0,0,0,0,0,0
3.0,58.0,1.0,0,0,0,0,0,0,0,0,...,36.0,0.305085,0,0,0,0,0,0,0,0


In [23]:
#save to file
hemoComposite.to_csv("../Data Validation/ARIC/Original DataFrames/HemoAricCohort.csv")

In [24]:
#MAke Hemo Labels
labels = pd.DataFrame()
labels['ID'] = hemoComposite.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hemoComposite.index:
    pt = hfchfaps16.loc[idx]
    pt1 = hfccelps16.loc[idx]
    if pt1['CELB06'] == 'Y' or pt1['CELB12'] == 'Y' or pt['HFAA57'] == 'Y' or pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

NameError: name 'hfchfaps16' is not defined

In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/HemoLabelsAricCohort.csv")

In [ ]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

In [ ]:
#save to file
allData.to_csv("../Data Validation/ARIC/Original DataFrames/AllDataAricCohort.csv")

In [ ]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = hfcoccps16.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    pt1 = hfccelps16.loc[idx]
    if pt1['CELB06'] == 'Y' or pt1['CELB12'] == 'Y' or pt['HFAA57'] == 'Y' or pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/LabelsAricCohort.csv")